<a href="https://colab.research.google.com/github/prathamnishad/experimental/blob/master/internship_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
from datetime import datetime
import tensorflow as tf
from tensorflow.python.data import Dataset
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers
from sklearn.model_selection import train_test_split



local_zip = '/content/drive/My Drive/raw.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/data')
raw=os.path.join("/tmp/data/raw")
train_gyro_phone=os.path.join("/tmp/data/raw/train/phone/gyro")
print('total train folder:', len(os.listdir(train_gyro_phone)))
train_accel_phone=os.path.join("/tmp/data/raw/train/phone/accel")
print('total train folder:', len(os.listdir(train_accel_phone)))
train_gyro_watch=os.path.join("/tmp/data/raw/train/watch/gyro")
print('total train folder:', len(os.listdir(train_gyro_watch)))
train_accel_watch=os.path.join("/tmp/data/raw/train/watch/accel")
print('total train folder:', len(os.listdir(train_accel_watch)))

test_gyro_phone=os.path.join("/tmp/data/raw/test/phone/gyro")
print('total test folder:', len(os.listdir(test_gyro_phone)))
test_accel_phone=os.path.join("/tmp/data/raw/test/phone/accel")
print('total test folder:', len(os.listdir(test_accel_phone)))
test_gyro_watch=os.path.join("/tmp/data/raw/test/watch/gyro")
print('total test folder:', len(os.listdir(test_gyro_watch)))
test_accel_watch=os.path.join("/tmp/data/raw/test/watch/accel")
print('total test folder:', len(os.listdir(test_accel_watch)))


Using TensorFlow backend.


total train folder: 20
total train folder: 20
total train folder: 20
total train folder: 20
total test folder: 14
total test folder: 14
total test folder: 14
total test folder: 14


In [14]:
#for train
df=pd.DataFrame()
for n in os.listdir(train_gyro_phone):
    a=pd.read_csv(os.path.join(train_gyro_phone,n),sep=",",header=None)
    df=pd.concat([df,a])
for n in os.listdir(train_accel_phone):
    a=pd.read_csv(os.path.join(train_accel_phone,n),sep=",",header=None)
    df=pd.concat([df,a])
  
for n in os.listdir(train_gyro_watch):
    a=pd.read_csv(os.path.join(train_gyro_watch,n),sep=",",header=None)
    df=pd.concat([df,a])

for n in os.listdir(train_accel_watch):
    a=pd.read_csv(os.path.join(train_accel_watch,n),sep=",",header=None)
    df=pd.concat([df,a])
d_train=df
df.head()

,0,1,2,3,4,5
0,1612,A,3952189047449,0.817322,-5.137268,-0.20063782;
1,1612,A,3952239401453,-1.029923,-3.979431,-1.3396301;
2,1612,A,3952289755457,0.493454,-1.189850,-2.666687;
3,1612,A,3952340109460,2.300003,-2.157044,-1.5438995;
4,1612,A,3952390463464,1.797806,-2.138672,1.0609283;


In [3]:
#for test
df_test=pd.DataFrame()
for n in os.listdir(test_gyro_phone):
    a=pd.read_csv(os.path.join(test_gyro_phone,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])
for n in os.listdir(test_accel_phone):
    a=pd.read_csv(os.path.join(test_accel_phone,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])
  
for n in os.listdir(test_gyro_watch):
    a=pd.read_csv(os.path.join(test_gyro_watch,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])

for n in os.listdir(test_accel_watch):
    a=pd.read_csv(os.path.join(test_accel_watch,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])
d_test=df_test
df_test.head()

,0,1,2,3,4,5
0,1626,A,598597359905477,1.646820,-1.594910,-0.08288574;
1,1626,A,598597410259481,0.910294,-1.438339,-0.18916321;
2,1626,A,598597460613485,0.264145,-0.924652,-0.39945984;
3,1626,A,598597510967489,0.193878,-0.760834,-0.28804016;
4,1626,A,598597561321493,0.107544,-0.810928,0.11076355;


In [0]:
def make_df(df):
  df.columns=["id","activity","time","x","y","z"]
  df['z'] = df['z'].apply(lambda x: float(x[:-1]))
  df['time'] = df['time'].apply(lambda x: x//1000000)
  df['time'] = df['time'].apply(lambda x: datetime.fromtimestamp(x))
  for n in ("month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"):
      df[n]=getattr(df.time.dt,n)
  df.drop(columns="time",inplace=True)

  for n in ["is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]:
    df[n]=df[n].apply(lambda x:int(x))

  act=["A","B","C","D","E","F","G","H","I","J","K","L","M","O","P","Q","R","S"]
  df["activity"]=df["activity"].apply(lambda a: act.index(a))
  df = df.sample(frac=1).reset_index(drop=True)
  df.drop(columns="id",inplace=True)
  print(df.head())
  return df
  

In [16]:
df_train=make_df(df)

   activity         x         y  ...  is_quarter_start  is_year_end  is_year_start
0        12  0.991653  0.024109  ...                 0            0              0
1        13 -1.395144 -2.898368  ...                 0            0              0
2         9 -0.012783 -0.001598  ...                 0            0              0
3         6 -4.195240  1.026514  ...                 0            0              0
4         6 -0.010727 -0.009521  ...                 0            0              0

[5 rows x 15 columns]


In [17]:
#for test
df_test=make_df(df_test)

   activity         x         y  ...  is_quarter_start  is_year_end  is_year_start
0        15 -0.502133 -5.735318  ...                 0            0              0
1        16 -5.949219 -3.750610  ...                 0            0              0
2         3  4.574539 -5.193070  ...                 0            0              0
3        10  0.467695  0.071197  ...                 0            0              0
4         9  3.853455 -5.595276  ...                 0            0              0

[5 rows x 15 columns]


In [18]:
df_train.describe()

,activity,x,y,z,month,week,day,dayofweek,dayofyear,is_month_end,is_month_start,is_quarter_end,is_quarter_start,is_year_end,is_year_start
count,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06,5.575010e+06
mean,8.546896e+00,2.479627e-01,-7.884989e-01,1.184300e+00,5.718961e+00,2.314056e+01,1.546356e+01,2.994202e+00,1.582390e+02,3.337842e-02,3.332281e-02,1.163693e-02,1.133774e-02,2.815242e-03,2.557843e-03
std,5.170262e+00,4.523955e+00,5.097889e+00,3.844963e+00,2.913113e+00,1.262981e+01,8.780354e+00,1.999477e+00,8.901066e+01,1.796227e-01,1.794781e-01,1.072451e-01,1.058735e-01,5.298412e-02,5.051040e-02
min,0.000000e+00,-3.981481e+01,-3.497829e+01,-3.454896e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,-1.103836e+00,-2.827552e+00,-1.929779e-01,3.000000e+00,1.300000e+01,8.000000e+00,1.000000e+00,8.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,9.000000e+00,-3.555298e-03,-1.724676e-02,2.427724e-02,6.000000e+00,2.400000e+01,1.500000e+01,3.000000e+00,1.650000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.300000e+01,1.102074e+00,3.547516e-01,2.685247e+00,8.000000e+00,3.300000e+01,2.300000e+01,5.000000e+00,2.240000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.700000e+01,3.489912e+01,5.235273e+01,3.024910e+01,1.200000e+01,5.300000e+01,3.100000e+01,6.000000e+00,3.660000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [32]:
df_test.describe()

,activity,x,y,z,month,week,day,dayofweek,dayofyear,is_month_end,is_month_start,is_quarter_end,is_quarter_start,is_year_end,is_year_start
count,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06,4.266331e+06
mean,8.483027e+00,1.622971e-01,-1.515056e+00,1.924461e-01,5.525555e+00,2.260227e+01,1.545530e+01,3.019399e+00,1.524881e+02,3.033121e-02,3.303049e-02,1.145645e-02,1.151411e-02,5.712168e-03,4.629505e-03
std,5.214518e+00,4.488122e+00,5.052591e+00,3.816337e+00,3.747188e+00,1.652288e+01,8.804018e+00,1.995365e+00,1.151397e+02,1.714971e-01,1.787162e-01,1.064199e-01,1.066843e-01,7.536273e-02,6.788279e-02
min,0.000000e+00,-3.491433e+01,-3.489500e+01,-3.498486e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,-1.278320e+00,-4.472526e+00,-8.988731e-01,3.000000e+00,9.000000e+00,8.000000e+00,1.000000e+00,6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+00,-3.814697e-04,-8.001148e-02,1.037301e-03,5.000000e+00,1.800000e+01,1.500000e+01,3.000000e+00,1.210000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.300000e+01,1.380947e+00,1.801455e-01,1.199645e+00,9.000000e+00,4.000000e+01,2.300000e+01,5.000000e+00,2.710000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.700000e+01,3.490542e+01,3.406907e+01,1.966373e+01,1.200000e+01,5.300000e+01,3.100000e+01,6.000000e+00,3.660000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [19]:
dfa=pd.DataFrame()
dfa=pd.concat([dfa,df_test])
dfa=pd.concat([dfa,df_train])
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
dfa[["x","y","z","month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]] = mm_scaler.fit_transform(dfa[["x","y","z","month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]])
dfa.describe()

,activity,x,y,z,month,week,day,dayofweek,dayofyear,is_month_end,is_month_start,is_quarter_end,is_quarter_start,is_year_end,is_year_start
count,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06
mean,8.519208e+00,5.356734e-01,3.878899e-01,5.478616e-01,4.213743e-01,4.212925e-01,4.819992e-01,5.008542e-01,4.239614e-01,3.205742e-02,3.319609e-02,1.155869e-02,1.141420e-02,4.071092e-03,3.455931e-03
std,5.189590e+00,6.034050e-02,5.829600e-02,5.923248e-02,3.001880e-01,2.778723e-01,2.930207e-01,3.329557e-01,2.772879e-01,1.761526e-01,1.791483e-01,1.068882e-01,1.062258e-01,6.367510e-02,5.868550e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,5.170561e-01,3.606420e-01,5.298223e-01,1.818182e-01,1.923077e-01,2.333333e-01,1.666667e-01,1.945205e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,9.000000e+00,5.328251e-01,4.001552e-01,5.364136e-01,3.636364e-01,3.846154e-01,4.666667e-01,5.000000e-01,3.863014e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.300000e+01,5.492285e-01,4.036007e-01,5.657928e-01,6.363636e-01,6.346154e-01,7.333333e-01,8.333333e-01,6.356164e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.700000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [0]:

train_y=dfa["activity"]
train_x=dfa.drop(columns="activity")
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y , train_size = 0.7, random_state =  90)


# model = keras.Sequential([
#  keras.layers.Dense(256, activation=tf.nn.relu,                  
#  input_shape=(x_train.shape[1],)),
#  keras.layers.Dense(128, activation=tf.nn.relu),
#  keras.layers.Dense(128, activation=tf.nn.relu),
#  keras.layers.Dense(64, activation=tf.nn.relu),
#  keras.layers.Dense(18, activation=  'softmax')
#  ])

# model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# history1 = model.fit(
#  x_train, y_train,
#  epochs= 15, batch_size = 128,
#  validation_data = (x_test, y_test))

In [21]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 50)
classifier.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
y_pred = classifier.predict(x_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Accuracy: 0.9349543405829083


In [23]:
from sklearn.externals import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(classifier, '/content/drive/My Drive/intern.pkl') 
  


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['/content/drive/My Drive/intern.pkl']

In [0]:
# Load the model from the file 
knn_from_joblib = joblib.load('/content/drive/My Drive/intern.pkl'  
  
# Use the loaded model to make predictions 
knn_from_joblib.predict(x_test) 